In [29]:
from gurobipy import Model, GRB, quicksum

In [30]:
# Create a new model
m = Model("SunnyshoreBay")

In [31]:
# Initial data from the problem
initial_cash = 140000
revenues = [180000, 260000, 420000, 580000]  # Revenues for May, June, July, August
expenses = [300000, 400000, 350000, 200000]  # Expenses for May, June, July, August
interest_rates = [0.0175, 0.0225, 0.0275]    # Interest rates for 1-month, 2-months, 3-months
borrow_limits = [250000, 150000, 350000]      # Borrowing limits for May, June, July

In [32]:
# Decision variables
# B[i,j] for borrowing amount in month i to be repaid in month j
B = {(i, j): m.addVar(name=f"B_{i+1},{j+1}") for i in range(4) for j in range(i, 4)}

In [33]:
# w[i] for cash balance at the end of each month i
w = {i: m.addVar(name=f"w_{i+1}") for i in range(4)}

In [34]:
# Objective function: Minimize the total amount repaid to the bank over the summer months
m.setObjective(
    (B[0, 0] * (1 + interest_rates[0])) + 
    (B[0, 1] * (1 + interest_rates[1])) + 
    (B[0, 2] * (1 + interest_rates[2])) + 
    (B[1, 1] * (1 + interest_rates[0])) + 
    (B[1, 2] * (1 + interest_rates[1])) + 
    (B[2, 2] * (1 + interest_rates[0])),
    GRB.MINIMIZE
)


In [35]:
# Constraints

# Cash balance constraints
cash_balance_constraints = [25000, 20000, 35000, 18000]
for i in range(4):
    m.addConstr(w[i] >= cash_balance_constraints[i], name=f"cash_balance_{i+1}")

In [36]:
# Cash balance at the end of July must be at least 65% of the combined total cash balances from May and June
m.addConstr(w[2] >= 0.65 * (w[0] + w[1]), name="July_cash_balance")

<gurobi.Constr *Awaiting Model Update*>

In [37]:
# Borrowing limits
m.addConstrs((quicksum(B[i, j] for j in range(i, 4)) <= borrow_limits[i] for i in range(3)), name="borrowing_limits")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

In [38]:
# Cash flow constraints
# May
m.addConstr(w[0] == initial_cash + revenues[0] - expenses[0] + B[0, 0] + B[0, 1] + B[0, 2], "cash_flow_May")
# June
m.addConstr(w[1] == w[0] + revenues[1] - expenses[1] + B[1, 1] + B[1, 2] - (B[0, 0] * (1 + interest_rates[0])), "cash_flow_June")
# July
m.addConstr(w[2] == w[1] + revenues[2] - expenses[2] + B[2, 2] - (B[0, 1] * (1 + interest_rates[1])) - (B[1, 1] * (1 + interest_rates[0])), "cash_flow_July")
# August
m.addConstr(w[3] == w[2] + revenues[3] - expenses[3] - (B[0, 2] * (1 + interest_rates[2])) - (B[1, 2] * (1 + interest_rates[1])) - (B[2, 2] * (1 + interest_rates[0])), "cash_flow_August")


<gurobi.Constr *Awaiting Model Update*>

In [39]:
# Non-negativity constraints
for i in range(4):
    for j in range(i, 4):
        m.addConstr(B[i, j] >= 0, name=f"B_nonneg_{i+1},{j+1}")
    m.addConstr(w[i] >= 0, name=f"w_nonneg_{i+1}")

In [40]:
# Solve the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26 rows, 14 columns and 49 nonzeros
Model fingerprint: 0xa6e704dd
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 19 rows and 5 columns
Presolve time: 0.00s
Presolved: 7 rows, 9 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.812500e+04   0.000000e+00      0s
       6    1.4290473e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.429047297e+05


In [41]:
# Output results
if m.status == GRB.OPTIMAL:
    print(f"Objective Value: ${m.ObjVal}")
    for v in m.getVars():
        print(f"{v.VarName} = {v.X}")

else:
    print("No optimal solution found")

Objective Value: $142904.72972972973
B_1,1 = 0.0
B_1,2 = 0.0
B_1,3 = 5000.0
B_1,4 = 0.0
B_2,2 = 54054.05405405405
B_2,3 = 80945.94594594595
B_2,4 = 0.0
B_3,3 = 0.0
B_3,4 = 0.0
B_4,4 = 0.0
w_1 = 25000.0
w_2 = 20000.0
w_3 = 35000.0
w_4 = 327095.2702702703


In [42]:
# Remove the old constraint for June's cash balance if it exists
m.remove(m.getConstrByName('cash_balance_2'))

# Add the new constraint with the updated cash balance for June
m.addConstr(w[1] >= 27500, name="cash_balance_June_updated")

# Resolve the model with the updated constraint
m.update()
m.optimize()

# Output the new total repayment amount
if m.status == GRB.OPTIMAL:
    print(f"Updated Objective Value: ${m.ObjVal}")
else:
    print("No optimal solution found with the updated constraints")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26 rows, 14 columns and 49 nonzeros
Model fingerprint: 0x82f949b7
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 19 rows and 5 columns
Presolve time: 0.00s
Presolved: 7 rows, 9 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.906250e+04   0.000000e+00      0s
       6    1.5053662e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.505366247e+05
Updated Objective Value: $150536.6246928747


In [45]:
from gurobipy import Model, GRB, quicksum

# Assumed initial data
cash_balance_constraints = [25000, 27500, 35000, 18000]  # Updated June constraint
borrow_limits = [250000, 150000, 350000]
interest_rates = [0.0175, 0.0225, 0.0275]

# Create the dual model
dual = Model("SunnyshoreBayDual")

# Dual variables for the cash balance constraints
pi = {i: dual.addVar(name=f"pi_{i+1}") for i in range(4)}

# Dual variables for the borrowing limits
lambda_ = {i: dual.addVar(name=f"lambda_{i+1}") for i in range(3)}

# The objective is to maximize the utility of cash balance and borrowing limits
dual.setObjective(
    quicksum(pi[i] * cash_balance_constraints[i] for i in range(4)) +
    quicksum(lambda_[i] * borrow_limits[i] for i in range(3)),
    GRB.MAXIMIZE
)

# Constraints for the dual problem are derived from the primal variables
# For each primal variable B[i, j], we need to create a constraint in the dual
# May borrowings
for j in range(3):
    dual.addConstr(
        quicksum(pi[i] for i in range(j+1)) - 
        lambda_[j] <= 
        (1 + interest_rates[j]),
        name=f"dual_constraint_May_{j+1}"
    )

# June borrowings
dual.addConstr(
    pi[2] + pi[3] - lambda_[1] <= 
    (1 + interest_rates[0]),
    name="dual_constraint_June_3"
)

dual.addConstr(
    pi[3] - lambda_[1] <= 
    (1 + interest_rates[1]),
    name="dual_constraint_June_4"
)

# July borrowing
dual.addConstr(
    pi[3] - lambda_[2] <= 
    (1 + interest_rates[0]),
    name="dual_constraint_July_4"
)

# Solving the dual
dual.optimize()

# Output results for the dual
if dual.status == GRB.OPTIMAL:
    print(f"Objective Value (Dual): ${dual.ObjVal}")
    for v in dual.getVars():
        print(f"{v.VarName} = {v.X}")
else:
    print("No optimal solution found for the dual problem")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 7 columns and 16 nonzeros
Model fingerprint: 0x0e0adb09
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible or unbounded model
No optimal solution found for the dual problem
